### Package Installation

In [ ]:
pip install -U datasets huggingface_hub fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.2/484.2 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.31.1
    Uninstalling huggingface-hub-0.31.1:
      Successfully uninstalled huggingface-hub-0.31.1
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_s

In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=5fbc0ee24cbaee2bf2958ef3efc7ba09c772495b49c2c1fc3eeb06ae5819ea12
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [ ]:
!pip install --upgrade datasets

In [ ]:
%pip install transformers==4.44.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 29.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


### Importing necessary Libraries

In [ ]:
from datasets import load_dataset, ClassLabel
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import numpy as np
import torch
from seqeval.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score
from google.colab import userdata

### Loading Data & Preprocessing

In [ ]:


dataset = load_dataset("surrey-nlp/PLODv2-filtered")

README.md:   0%|          | 0.00/295 [00:00<?, ?B/s]

PLOD_v2_filtered_train.parquet:   0%|          | 0.00/25.0M [00:00<?, ?B/s]

PLOD_v2_filtered_val.parquet:   0%|          | 0.00/3.75M [00:00<?, ?B/s]

PLOD_v2_filtered_test.parquet:   0%|          | 0.00/3.75M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/112652 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/24140 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/24140 [00:00<?, ? examples/s]

In [ ]:


label_list = list(set(tag for instance in dataset["train"]["ner_tags"] for tag in instance))
print(label_list)
num_labels = len(label_list)
num_labels

label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for i, label in enumerate(label_list)}



['B-AC', 'I-LF', 'O', 'B-LF']


In [ ]:


def encode_labels(example):
  example["ner_tags"] = [label2id[label] for label in example["ner_tags"]]
  return example

encoded_dataset = dataset.map(encode_labels)

Map:   0%|          | 0/112652 [00:00<?, ? examples/s]

Map:   0%|          | 0/24140 [00:00<?, ? examples/s]

Map:   0%|          | 0/24140 [00:00<?, ? examples/s]

### Loading Tokenizer and Model

In [ ]:


model_checkpoint = "dmis-lab/biobert-base-cased-v1.2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

model.config.id2label = id2label
model.config.label2id = label2id


config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:


def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(examples["tokens"], truncation=True, max_length=512, padding="max_length", is_split_into_words=True)
  labels = []
  for i, label in enumerate(examples["ner_tags"]):
    word_ids = tokenized_inputs.word_ids(batch_index=i)
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
      if word_idx is None:
        label_ids.append(-100)
      elif word_idx != previous_word_idx:
        label_ids.append(label[word_idx])
      else:
        label_ids.append(label[word_idx])
      previous_word_idx = word_idx
    labels.append(label_ids)
  tokenized_inputs["labels"] = labels
  return tokenized_inputs





In [ ]:


tokenized_datasets = encoded_dataset.map(tokenize_and_align_labels, batched=True)
data_collator = DataCollatorForTokenClassification(tokenizer)

Map:   0%|          | 0/112652 [00:00<?, ? examples/s]

Map:   0%|          | 0/24140 [00:00<?, ? examples/s]

Map:   0%|          | 0/24140 [00:00<?, ? examples/s]

In [ ]:


def compute_metrics(p):
  predictions, labels = p
  predictions = np.argmax(predictions, axis=2)

  true_labels = [[l for l in label if l != -100] for label in labels]
  true_preds = [[p for (p,l) in zip(pred, label) if l != -100] for pred, label in zip(predictions, labels)]

  precision, recall, f1, _ = precision_recall_fscore_support(
      [item for sublist in true_labels for item in sublist],
      [item for sublist in true_preds for item in sublist],
      average='macro'
  )
  accuracy = accuracy_score(
      [item for sublist in true_labels for item in sublist],
      [item for sublist in true_preds for item in sublist]
  )
  return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}


## Training and Evaluating on Different Proportions




### Training with 15% of Training Data:

In [ ]:
proportion = 0.15

print(f"\nTraining with {int(proportion*100)}% of training data")


train_size = int(len(dataset["train"]) * proportion)
train_subset = tokenized_datasets["train"].select(range(train_size))
eval_size = int(len(dataset["validation"]) * proportion)
eval_subset = tokenized_datasets["validation"].select(range(eval_size))



training_args = TrainingArguments(
    output_dir=f"./results-{int(proportion * 100)}",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=eval_subset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)



trainer.train()
metrics = trainer.evaluate()
print(metrics)


Training with 15% of training data


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: u4859234 (u4859234-university-of-surrey) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.189200,0.178207,0.845257,0.908358,0.873799,0.928005
2,0.182900,0.166527,0.847440,0.922681,0.881962,0.931847


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.189200,0.178207,0.845257,0.908358,0.873799,0.928005
2,0.182900,0.166527,0.847440,0.922681,0.881962,0.931847
3,0.090900,0.178587,0.857101,0.913672,0.883648,0.933492


{'eval_loss': 0.17858664691448212, 'eval_precision': 0.8571005898776427, 'eval_recall': 0.9136717686654493, 'eval_f1': 0.8836484431831728, 'eval_accuracy': 0.9334923147327895, 'eval_runtime': 109.6182, 'eval_samples_per_second': 33.033, 'eval_steps_per_second': 4.133, 'epoch': 3.0}


In [ ]:
!nvidia-smi

Thu May 15 02:52:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   73C    P0             43W /   70W |    4078MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Evaluating Model Performance on Test Data

In [ ]:


print("\nEvaluating on test set...")

test_size = int(len(dataset["test"]) * proportion)
test_subset = tokenized_datasets["test"].select(range(test_size))
predictions, labels, _ = trainer.predict(test_subset)


pred_ids = np.argmax(predictions, axis=2)


true_labels = []
true_predictions = []

for prediction, label in zip(pred_ids, labels):
    true_label = []
    true_pred = []
    for p, l in zip(prediction, label):
        if l != -100:
            true_label.append(id2label[l])
            true_pred.append(id2label[p])
    true_labels.append(true_label)
    true_predictions.append(true_pred)


print("\nClassification Report on Test Set:\n")
print(classification_report(true_labels, true_predictions))

precision = precision_score(true_labels, true_predictions)
recall = recall_score(true_labels, true_predictions)
f1 = f1_score(true_labels, true_predictions)
accuracy = accuracy_score(true_labels, true_predictions)

print(f"\nFinal Test Metrics:")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1-score : {f1:.4f}")
print(f"Accuracy : {accuracy:.4f}")



Evaluating on test set...



Classification Report on Test Set:

              precision    recall  f1-score   support

          AC       0.84      0.89      0.86     25374
          LF       0.76      0.87      0.81     12262

   micro avg       0.81      0.88      0.85     37636
   macro avg       0.80      0.88      0.84     37636
weighted avg       0.81      0.88      0.85     37636


Final Test Metrics:
Precision: 0.8113
Recall   : 0.8847
F1-score : 0.8464
Accuracy : 0.9311
